# Scrapy 错误处理

In [ ]:

import scrapy


class CarSpider(scrapy.Spider):
    name = "car"
    allowed_domains = ["car.autohome.com.cn"]
    # 注意如果你的请求的接口是html结尾的，那么是不需要加/的
    start_urls = ["https://car.autohome.com.cn/price/brand-15.html"]

    def parse(self, response):
        # selector 列表
        name_list = response.xpath('//div[@class="main-title"]/a/text()')
        price_list = response.xpath('//div[@class="main-lever"]//span/span/text()')

        for i in range(len(name_list)):
            name = name_list[i].extract()
            price = price_list[i].extract()
            print(name, price)

    def errback(self, failure):
        # log all failures
        self.logger.error(repr(failure)) # repr 返回对象的字符串表示

In [ ]:
# scrapy 爬取多个网页的数据，有的失败了。

import scrapy

class PytorchApiDocSpider(scrapy.Spider):
    name = "pytorch_api_doc"
    allowed_domains = ["pytorch.org"]

    def start_requests(self):
        urls = [
            "https://pytorch.org/docs/2.4/generated/torch.bernoulli.html#torch.bernoulli",
            # 添加更多的URL
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse, errback=self.errback)

    def parse(self, response):
        # 解析成功的响应
        api_name = response.meta.get("api_name", "unknown")
        title = response.xpath("//h1/text()").get()
        self.log(f"Successfully parsed: {api_name} - {title}")

    def errback(self, failure):
        # 处理失败的响应
        self.log(f"Request failed: {failure.request.url}")
        self.log(f"Error: {failure.value}")